In [1]:
import pandas as pd

In [2]:
data_file = "../data/data.csv"

In [3]:
data = pd.read_csv(data_file)

In [4]:
data.size

840928

In [5]:
data.shape

(420464, 2)

In [6]:
data

,url,label
0,diaryofagameaddict.com,bad
1,espdesign.com.au,bad
2,iamagameaddict.com,bad
3,kalantzis.net,bad
4,slightlyoffcenter.net,bad
...,...,...
420459,23.227.196.215/,bad
420460,apple-checker.org/,bad
420461,apple-iclods.org/,bad
420462,apple-uptoday.org/,bad


In [7]:
data.describe()

,url,label
count,420464,420464
unique,411247,2
top,d11m2p9mpffp32.cloudfront.net/main/web_zt.exe,good
freq,27,344821


In [8]:
#去空格
num_url = data.shape[0]
for i in range(num_url):
    stri = data.iloc[i, 0]
    stri_nowhitespace = ''.join(stri.split())
    if not stri == stri_nowhitespace:
        data.iloc[i, 0] = stri_nowhitespace

In [9]:
count = 0
for url in data['url']:
    if url.startswith('http'):
        count = count + 1
        # print(url)
print(count)

123


In [11]:
#去协议头
for i in range(data.shape[0]):
    stri = data.iloc[i, 0] #replace(' ', '')
    if stri.startswith("http://"):
        #line=line[7:]
        data.iloc[i, 0] = stri[7:]
    if stri.startswith("https://"):
        #line=line[8:]
        data.iloc[i, 0] = stri[8:]
    if stri.startswith("ftp://"):
        #line=line[6:]
        data.iloc[i, 0] = stri[6:]
    if stri.startswith("www."):
        #line = line[4:]
        data.iloc[i, 0] = stri[4:]
    if stri.startswith("http:/"): #数据集中错误
        data.iloc[i, 0] = stri[6:]

In [12]:
count = 0
for url in data['url']:
    if url.startswith('http'):
        count = count + 1
        print(url)
print(count)

https-paypal.verifications-updates.com/
https.www.appleid.verify.account.user.login.auth.co.uk.acc93.co.uk/apple/
https.www.payypal.com.resolution.center.limitation.details-cases.services/au/webapps/maab/home
https.www.payppal.com.resolution.center.limitation.details-cases.services/signin
https.www.paypal.com.uk-cgi-bin.webscr.cmd-login-submit-dispatch.0af7806a38a1ba9d9cbac36ad9aeda70e1c.0d0146638fd6ae870e1c044a751271270af789147127071270af7af7806a23.a826112891471270af7807512891471270.givemefollowers.com/PP/PP/websc
https.www.paypal.com.uk-cgi-bin.webscr.cmd-login-submit-dispatch.0af7806a38a1ba9d9cbac36ad9aeda70e1c.0d0146638fd6ae870e1c044a751271270af789147127071270af7af7806a23.a826112891471270af7807512891471270.givemefollowers.com/PP/PP/?cmd=_home&amp;dispatch=588
https.www.paypal.com.uk-cgi-bin.webscr.cmd-login-submit-dispatch.0af7806a38a1ba9d9cbac36ad9aeda70e1c.0d0146638fd6ae870e1c044a751271270af789147127071270af7af7806a23.a826112891471270af7807512891471270.givemefollowers.com/PP/PP/?

In [13]:
#去末尾/
num_url = data.shape[0]
for i in range(num_url):
    stri = data.iloc[i, 0]
    if not stri.endswith('/'):
        data.iloc[i, 0] = stri[:-1]

In [ ]:
data.to_csv("../data/data_processed.csv", index=False)

In [23]:
#shuffle
data = data.sample(frac=1)

In [24]:
labels = [1 if data.iloc[i, 1] == 'bad' else -1 for i in range(420464)]

In [25]:
urls = data['url'].to_numpy()

In [26]:
urls

array(['itunes.apple.com/us/artist/clint-black/id21516',
       'terry-pruyne.suite101.com/joseph-campbell-hero-cycle-in-the-matrix-a29309',
       'raaga.com/channels/tamil/nonstop.as', ...,
       'baseball-almanac.com/teamstats/roster.php?y=1954&t=SL',
       'kinafreyr.com/trec4x?zULftMedhRE=hmaLeuhB', 'zh.com/'],
      dtype=object)

In [27]:
len(urls), len(labels)

(420464, 420464)

In [28]:
import sys
sys.path.append("..")

In [29]:
from utils import vectorization

In [30]:
#word level
word_vectorizer = vectorization.get_word_vectorizer(urls)

In [31]:
import pickle
with open('../model/word_vectorizer.pickle', 'wb') as f:
    pickle.dump(word_vectorizer, f)

In [48]:
data_x = word_vectorizer.transform(urls)

In [49]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_x, labels, test_size=0.2, random_state=42)

In [50]:
x_train

<336371x500 sparse matrix of type '<class 'numpy.int64'>'
	with 1729865 stored elements in Compressed Sparse Row format>

In [51]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(max_iter=3000)
lgs.fit(x_train, y_train)

LogisticRegression(max_iter=3000)

In [52]:
import pickle
with open('../model/lgs_word.pickle', 'wb') as f:
    pickle.dump(lgs, f)

In [53]:
lgs.score(x_test, y_test)

0.9331216629208139

In [58]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

          -1       0.94      0.99      0.96     68809
           1       0.91      0.70      0.79     15284

    accuracy                           0.93     84093
   macro avg       0.92      0.84      0.88     84093
weighted avg       0.93      0.93      0.93     84093



In [59]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

array([[67799,  1010],
       [ 4641, 10643]], dtype=int64)

In [60]:
from sklearn.metrics import confusion_matrix
y_pred = lgs.predict(x_test)
y_true = y_test
confusion_matrix(y_true, y_pred)

array([[67766,  1043],
       [ 4581, 10703]], dtype=int64)

In [63]:
from sklearn.svm import LinearSVC

clf = LinearSVC(max_iter=5000)

In [64]:
clf.fit(x_train, y_train)

LinearSVC(max_iter=5000)

In [65]:
clf.score(x_test, y_test)

0.9328243730155898

In [66]:
y_pred = clf.predict(x_test)
y_true = y_test
confusion_matrix(y_true, y_pred)

array([[67799,  1010],
       [ 4639, 10645]], dtype=int64)

In [67]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

          -1       0.94      0.99      0.96     68809
           1       0.91      0.70      0.79     15284

    accuracy                           0.93     84093
   macro avg       0.92      0.84      0.88     84093
weighted avg       0.93      0.93      0.93     84093



In [79]:
with open('../model/linearsvc_word.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [80]:
#char level
from sklearn.feature_extraction.text import CountVectorizer
char_vectorizer = CountVectorizer(analyzer="char")

In [82]:
X = char_vectorizer.fit_transform(urls)

In [83]:
import pickle
with open('../model/char_vectorizer.pickle', 'wb') as f:
    pickle.dump(word_vectorizer, f)

In [84]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [85]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(max_iter=3000)
lgs.fit(x_train, y_train)

LogisticRegression(max_iter=3000)

In [86]:
lgs.score(x_test, y_test)

0.867313569500434

In [87]:
y_pred = lgs.predict(x_test)
y_true = y_test
confusion_matrix(y_true, y_pred)

array([[67131,  1678],
       [ 9480,  5804]], dtype=int64)

In [88]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

          -1       0.88      0.98      0.92     68809
           1       0.78      0.38      0.51     15284

    accuracy                           0.87     84093
   macro avg       0.83      0.68      0.72     84093
weighted avg       0.86      0.87      0.85     84093



In [89]:
import pickle
with open('../model/lgs_char.pickle', 'wb') as f:
    pickle.dump(lgs, f)

In [90]:
from sklearn.svm import LinearSVC

clf = LinearSVC(max_iter=5000)

In [96]:
clf.fit(x_train, y_train)

LinearSVC(max_iter=5000)

In [97]:
clf.score(x_test, y_test)

0.8645309359875376

In [98]:
y_pred = clf.predict(x_test)
y_true = y_test
confusion_matrix(y_true, y_pred)

array([[67650,  1159],
       [10233,  5051]], dtype=int64)

In [99]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

          -1       0.87      0.98      0.92     68809
           1       0.81      0.33      0.47     15284

    accuracy                           0.86     84093
   macro avg       0.84      0.66      0.70     84093
weighted avg       0.86      0.86      0.84     84093



In [100]:
with open('../model/linearsvc_char.pickle', 'wb') as f:
    pickle.dump(clf, f)